# Using `kugupu` to calculate molecular coupling networks

This notebook demonstrates how to calculate molecular coupling between fragments, inspect the results and save and load these results to file.  These results files will be the basis of all further analysis done using the `kugupu` package.

This will require version 0.20.0 of MDAnalysis, and kugupu to be installed.

In [21]:
import MDAnalysis as mda
import kugupu as kgp

Firstly we create an `MDAnalysis.Universe` object from our simulation files:

In [22]:
u = mda.Universe('datafiles/C6.data', 'datafiles/C6.dcd')

This system has 46,500 atoms in 250 different fragments.

In [23]:
print(u.atoms.n_atoms, len(u.atoms.fragments))

46500 250


Our dynamics simulation has 5 frames of results.

In [24]:
print(u.trajectory.n_frames)

5


To perform the coupling calculations our `Universe` will require bond information (for determining fragments) and element information (for the tight binding calculations) stored inside the `.names` attribute.

In [25]:
u.atoms.bonds

<TopologyGroup containing 50000 bonds>

Our Lammps Data file did not include element symbols, so we can add these to the Universe now...

In [26]:
def add_names(u):
    # Guesses atom names based upon masses
    def approx_equal(x, y):
        return abs(x - y) < 0.1
    
    # mapping of atom mass to element
    massdict = {}
    for m in set(u.atoms.masses):
        for elem, elem_mass in mda.topology.tables.masses.items():
            if approx_equal(m, elem_mass):
                massdict[m] = elem
                break
        else:
            raise ValueError
            
    u.add_TopologyAttr('names')
    for m, e in massdict.items():
        u.atoms[u.atoms.masses == m].names = e

add_names(u)

## Running the coupling matrix calculation

The coupling matrix between fragments is calculated using the `kgp.coupling_matrix` function.

Here we are calculating the coupling matrix for fragments in the Universe `u` where
- coupling is calculated between fragments with a closest approach of less than 5.0 Angstrom (`nn_cutoff`)
- coupling is calculated between the LUMO upwards (`state='lumo'`)
- one state per fragment is considered (`degeneracy=1`)
- we will analyse up to frame 3 (`stop=3`)

This function will (for each frame)
- identify which fragments are close enough to possibly be electronically coupled
- run a tight binding calculation between all pairs identified
- calculate the molecular coupling based on this tight binding calculation

In [27]:
res = kgp.coupling_matrix(u, nn_cutoff=5.0, state='lumo', degeneracy=1, stop=3)

2019-08-09T11:19:36.846494+0100 INFO Processing 3 frames
2019-08-09T11:19:37.714351+0100 INFO Processing frame 1 of 3
2019-08-09T11:19:38.778211+0100 INFO Finding dimers within 5.0, passed 250 fragments
2019-08-09T11:19:40.802750+0100 INFO Found 3282 dimers

 10%|█         | 339/3282 [00:20<02:55, 16.75it/s]


 25%|██▍       | 808/3282 [00:39<01:43, 23.91it/s]


 40%|████      | 1321/3282 [00:57<01:01, 31.96it/s]


 58%|█████▊    | 1889/3282 [01:16<00:45, 30.78it/s]


 74%|███████▍  | 2431/3282 [01:34<00:28, 29.45it/s]


...

 37%|███▋      | 1188/3254 [00:42<01:03, 32.67it/s]


 55%|█████▍    | 1780/3254 [01:01<00:46, 31.82it/s]


 73%|███████▎  | 2371/3254 [01:19<00:29, 30.43it/s]


 87%|████████▋ | 2838/3254 [01:37<00:13, 30.15it/s]


100%|██████████| 3254/3254 [01:50<00:00, 29.35it/s]2019-08-09T11:25:36.841239+0100 INFO Done!


The `res` object is a namedtuple which contains all the data necessary to perform further analysis.
This object has various attributes which will not be briefly explained.

The `.frames` attribute records which frames from the trajectory were analysed.
This is useful to later cross reference data with the original MD trajectory data.

In [28]:
print(res.frames)

[0 1 2]


The `.degeneracy` attribute stores how many degenerate states were considered for each fragment.
This value will not change over time, so this array has shape `nfragments`.

In this example only a single state per fragment was considered. 

In [29]:
print(res.degeneracy)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


The `.H_frag` attribute contains the molecular coupling values, stored inside a 3d numpy array.
The first dimension is along the number of frames (quasi time axis),
while the other two move along fragments in the system.

For example `res.H_frag[0, 1, 71]` gives the coupling (in eV) between the 2nd and 13th fragments in the first frame.

In [43]:
print(res.H_frag.shape)

print(res.H_frag[0, 1, 71])

(3, 250, 250)
0.03601076420417074


Producing these results is often a time consuming part of the analysis,
therefore it is wise to save them to a file so you can come back to them later!

This can be done using the `kugupu.save_results` function, which will save the results to a hdf5 (compressed) format.
These results can later be retrieved using the `kugupu.load_results` function.

In [13]:
kgp.save_results('myresults.hdf5', res)

These results can then be retrieved again using the `kugupu.load_results` function:

In [14]:
kgp.load_results('./myresults.hdf5')

KugupuResults(frames=array([0, 2, 4]), H_frag=array([[[-1.02815068e+01,  6.39656998e-03,  3.08628294e-01,
          1.86203671e-01],
        [ 6.39656998e-03, -1.02544681e+01,  0.00000000e+00,
          9.11791591e-03],
        [ 3.08628294e-01,  0.00000000e+00, -1.02588300e+01,
          2.11536686e-02],
        [ 1.86203671e-01,  9.11791591e-03,  2.11536686e-02,
         -1.02635364e+01]],

       [[-1.01967476e+01,  6.20170141e-04,  1.61034370e-01,
          9.60007450e-02],
        [ 6.20170141e-04, -1.02250729e+01,  0.00000000e+00,
          1.56797927e-07],
        [ 1.61034370e-01,  0.00000000e+00, -1.03186597e+01,
          1.29442219e-02],
        [ 9.60007450e-02,  1.56797927e-07,  1.29442219e-02,
         -1.02907822e+01]],

       [[-1.03943337e+01,  6.94727940e-02,  3.81707480e-04,
          1.05304069e-02],
        [ 6.94727940e-02, -1.02316326e+01,  0.00000000e+00,
          3.47187057e-03],
        [ 3.81707480e-04,  0.00000000e+00, -1.03398656e+01,
          6.32912880